#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [10]:
model.add(Dropout(rate=0.25))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='softmax'))

In [20]:
#檢查model 的STACK
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

'\n作業:\n請分別選用 "MSE", "binary _crossentropy"\n查看Train/test accurancy and loss rate\n'

In [24]:
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 9s 241us/step - loss: 2.1651 - accuracy: 0.2018 - val_loss: 2.0543 - val_accuracy: 0.2970
Epoch 2/12
37500/37500 [==============================] - 6s 154us/step - loss: 1.9510 - accuracy: 0.2996 - val_loss: 1.9531 - val_accuracy: 0.3367
Epoch 3/12
37500/37500 [==============================] - 6s 157us/step - loss: 1.8246 - accuracy: 0.3508 - val_loss: 1.8227 - val_accuracy: 0.3960
Epoch 4/12
37500/37500 [==============================] - 6s 157us/step - loss: 1.7223 - accuracy: 0.3861 - val_loss: 1.7420 - val_accuracy: 0.4198
Epoch 5/12
37500/37500 [==============================] - 6s 156us/step - loss: 1.6383 - accuracy: 0.4163 - val_loss: 1.6555 - val_accuracy: 0.4530
Epoch 6/12
37500/37500 [==============================] - 6s 153us/step - loss: 1.5686 - accuracy: 0.4372 - val_loss: 1.6097 - val_accuracy: 0.4687
Epoch 7/12
37500/37500 [==============================] - 6s 1

In [26]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print()
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 1s 108us/step

accuracy= 0.5472000241279602
loss= 1.3721178833007812


In [27]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

In [28]:
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 6s 172us/step - loss: 0.0605 - accuracy: 0.5336 - val_loss: 0.0628 - val_accuracy: 0.5467
Epoch 2/12
37500/37500 [==============================] - 6s 161us/step - loss: 0.0603 - accuracy: 0.5329 - val_loss: 0.0627 - val_accuracy: 0.5468
Epoch 3/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.0603 - accuracy: 0.5341 - val_loss: 0.0626 - val_accuracy: 0.5477
Epoch 4/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.0601 - accuracy: 0.5359 - val_loss: 0.0627 - val_accuracy: 0.5470
Epoch 5/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.0601 - accuracy: 0.5348 - val_loss: 0.0626 - val_accuracy: 0.5485
Epoch 6/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.0600 - accuracy: 0.5346 - val_loss: 0.0625 - val_accuracy: 0.5490
Epoch 7/12
37500/37500 [==============================] - 6s 1

In [29]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print()
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 1s 103us/step

accuracy= 0.5533999800682068
loss= 0.06188971630334854


In [30]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [31]:
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 6s 171us/step - loss: 0.2025 - accuracy: 0.9207 - val_loss: 0.2119 - val_accuracy: 0.9154
Epoch 2/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.2018 - accuracy: 0.9209 - val_loss: 0.2118 - val_accuracy: 0.9156
Epoch 3/12
37500/37500 [==============================] - 6s 159us/step - loss: 0.2012 - accuracy: 0.9210 - val_loss: 0.2110 - val_accuracy: 0.9160
Epoch 4/12
37500/37500 [==============================] - 6s 161us/step - loss: 0.2009 - accuracy: 0.9215 - val_loss: 0.2108 - val_accuracy: 0.9161
Epoch 5/12
37500/37500 [==============================] - 6s 162us/step - loss: 0.2010 - accuracy: 0.9210 - val_loss: 0.2102 - val_accuracy: 0.9163
Epoch 6/12
37500/37500 [==============================] - 6s 162us/step - loss: 0.2008 - accuracy: 0.9211 - val_loss: 0.2103 - val_accuracy: 0.9160
Epoch 7/12
37500/37500 [==============================] - 6s 1

In [32]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print()
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 1s 104us/step

accuracy= 0.9171800017356873
loss= 0.20653475830554963
